1. Pobierz bazę danych dotyczącą śmiertelnych interwencji policji w USA, a następnie wczytaj ją do obiektu DataFrame.

In [19]:
import pandas as pd
crim = pd.read_csv('fatal-police-shootings-data.csv')
crim.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False


2. Przekształć tabelę w taki sposób, aby wskazywała zestawienie jednocześnie liczby ofiar interwencji według rasy (‘race’) oraz tego, czy wykazywały one oznaki choroby psychicznej (‘signs_of_mental_illness’).

In [20]:
crim.pivot_table(values='name', index='race',columns='signs_of_mental_illness', aggfunc=len)

signs_of_mental_illness,False,True
race,,
A,69,24
B,1109,183
H,741,157
N,62,14
O,36,11
W,1754,709


3. Za pomocą Map, Applymap lub Apply dodaj do tego zestawienia kolumnę wskazującą jaki odsetek ofiar interwencji wykazywało oznaki choroby psychicznej dla każdej z ras. Odpowiedz, która z nich charakteryzuje się największym odsetkiem znamion choroby psychicznej podczas interwencji.

In [21]:
import pandas as pd
crim2 = pd.DataFrame({'not_ill': [69,1109,741,62,36,1754],'ill': [24,183,157,14,11,709]}, index=['A','B','H','N','O','W'])
crim2

def perc(row):
    stat = row['ill']/(row['not_ill']+row['ill'])
    return stat
    
crim2['Perc'] = crim2.apply(lambda row: perc(row),axis=1)
crim2

,not_ill,ill,Perc
A,69,24,0.258065
B,1109,183,0.141641
H,741,157,0.174833
N,62,14,0.184211
O,36,11,0.234043
W,1754,709,0.287860


Odp. Największy odsetek znamion choroby psychicznej podczas śmiertelnych interwencji wykazywała rasa W - ponad 28% wszystkich o rasie W zabitych podczas interwencji wykazywało znamiona choroby psychicznej.

4. Dodaj kolumnę oznaczającą dzień tygodnia, w którym doszło do interwencji. Zlicz interwencje według odpowiedniego dnia tygodnia. Następnie stwórz wykres kolumnowy, tak aby dni tygodnia były uszeregowane od poniedziałku do niedzieli.

4.1 Dodanie kolumny oznaczającej dzień tygodnia

In [22]:
crim['date'] = pd.to_datetime(crim['date'])
crim['Day'] = crim['date'].dt.day_name()
crim.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,Day
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,Friday
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,Friday
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,Saturday
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,Sunday
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,Sunday


4.2 Zliczenie interwencji według dni tygodnia

In [23]:
from pandas.api.types import CategoricalDtype

cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
cat_type = CategoricalDtype(categories=cats, ordered=True)

cat_type = CategoricalDtype(categories=cats, ordered=True)
crim['Day'] = crim['Day'].astype(cat_type)

crim2 = crim.pivot_table(values='name', index='Day', aggfunc=len)
crim3 = crim2.rename(columns={'name':'amount'})
crim3

,amount
Day,
Monday,721
Tuesday,813
Wednesday,826
Thursday,805
Friday,756
Saturday,721
Sunday,748


4.3 Stworzenie wykresu kolumnowego z dniami tygodnia uszeregowanymi od poniedziałku do niedzieli

In [29]:
import pandas as pd
crim3.to_excel('crim3.xlsx', index=False)
writer = pd.ExcelWriter('crim3.xlsx', engine='xlsxwriter')
crim3.to_excel(writer)
writer.save()

In [30]:
writer = pd.ExcelWriter('crim3.xlsx', engine='xlsxwriter')
crim3.to_excel(writer, sheet_name='my_chart')
workbook = writer.book
worksheet = writer.sheets['my_chart']
chart = workbook.add_chart({'type':'column'})

def grab_series(crim3, sheet_name, colname, startcol=0, startrow=0):
    col_index = crim3.columns.tolist().index(colname)
    col_letter = chr(ord('@')+(col_index+2+startcol))
    first_row = startrow + 2
    last_row = startrow + 1 + len(crim2)
    return f"='{sheet_name}'!{col_letter}{first_row}:{col_letter}{last_row}"

In [31]:
chart.add_series({'values':grab_series(crim3,'my_chart','amount')})
chart.set_x_axis({
    'name': 'Monday   Tuesday   Wednesday   Thursday   Friday   Saturday   Sunday',
    'name_font': {'size': 9, 'bold': True},
    'num_font':  {'italic': True },
})
chart.set_legend({'none': True})
worksheet.insert_chart('F2', chart)

0

In [32]:
writer.save()
workbook.close()

5. Wczytaj do projektu dane dotyczące populacji w poszczególnych stanach USA (https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population) oraz dane dotyczące skrótów poszczególnych stanów (https://en.wikipedia.org/wiki/List_of_U.S._state_and_territory_abbreviations). Połącz te bazy danych w taki sposób, aby móc policzyć do ilu incydentów w bazie dotyczącej śmiertelnych interwencji doszło w przeliczeniu na 1000 mieszkańców każdego ze stanów.

5.1 Import z html danych populacji USA

In [35]:
popul = pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population', header=0)
popul[0].head()

,"Rank in states & territories, 2019","Rank in states & territories, 2010",State,"Population estimate, July 1, 2019[2]","Census population, April 1, 2010[3]","Percent change, 2010–2019[note 1]","Absolute change, 2010-2019","Total seats in the U.S. House of Representatives, 2013–2023","Estimated population per electoral vote, 2019[note 2]","Estimated population per House seat, 2019","Census population per House seat, 2010","Percent of the total U.S. population, 2018[note 3]"
0,1.0,1.0,California,39512223,37254523,6.1%,+2257700,53,718404,745514,702885,11.96%
1,2.0,2.0,Texas,28995881,25145561,15.3%,+3850320,36,763050,805441,698503,8.68%
2,3.0,4.0,Florida,21477737,18801310,14.2%,+2676427,27,740611,795472,696468,6.44%
3,4.0,3.0,New York,19453561,19378102,0.4%,+75459,27,670812,720502,717707,5.91%
4,5.0,6.0,Pennsylvania,12801989,12702379,0.8%,+99610,18,640099,711222,705715,3.87%


5.2 Import z html skrótów stanów USA

In [36]:
stat = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._state_and_territory_abbreviations', header=0)
stat[0].head()

,"Codes: ISO ISO 3166 codes (2-letter, 3-letter, and 3-digit codes from ISO 3166-1; 2+2-letter codes from ISO 3166-2) ANSI 2-letter and 2-digit codes from the ANSI standard INCITS 38:2009 (supersedes FIPS 5-2) USPS 2-letter codes used by the United States Postal Service USCG 2-letter codes used by the United States Coast Guard (bold red text shows differences between ANSI and USCG) Abbreviations: GPO Older variable-length official US Government Printing Office abbreviations AP Abbreviations from the AP Stylebook (bold red text shows differences between GPO and AP)","Codes: ISO ISO 3166 codes (2-letter, 3-letter, and 3-digit codes from ISO 3166-1; 2+2-letter codes from ISO 3166-2) ANSI 2-letter and 2-digit codes from the ANSI standard INCITS 38:2009 (supersedes FIPS 5-2) USPS 2-letter codes used by the United States Postal Service USCG 2-letter codes used by the United States Coast Guard (bold red text shows differences between ANSI and USCG) Abbreviations: GPO Older variable-length official US Government Printing Office abbreviations AP Abbreviations from the AP Stylebook (bold red text shows differences between GPO and AP).1","Codes: ISO ISO 3166 codes (2-letter, 3-letter, and 3-digit codes from ISO 3166-1; 2+2-letter codes from ISO 3166-2) ANSI 2-letter and 2-digit codes from the ANSI standard INCITS 38:2009 (supersedes FIPS 5-2) USPS 2-letter codes used by the United States Postal Service USCG 2-letter codes used by the United States Coast Guard (bold red text shows differences between ANSI and USCG) Abbreviations: GPO Older variable-length official US Government Printing Office abbreviations AP Abbreviations from the AP Stylebook (bold red text shows differences between GPO and AP).2","Codes: ISO ISO 3166 codes (2-letter, 3-letter, and 3-digit codes from ISO 3166-1; 2+2-letter codes from ISO 3166-2) ANSI 2-letter and 2-digit codes from the ANSI standard INCITS 38:2009 (supersedes FIPS 5-2) USPS 2-letter codes used by the United States Postal Service USCG 2-letter codes used by the United States Coast Guard (bold red text shows differences between ANSI and USCG) Abbreviations: GPO Older variable-length official US Government Printing Office abbreviations AP Abbreviations from the AP Stylebook (bold red text shows differences between GPO and AP).3","Codes: ISO ISO 3166 codes (2-letter, 3-letter, and 3-digit codes from ISO 3166-1; 2+2-letter codes from ISO 3166-2) ANSI 2-letter and 2-digit codes from the ANSI standard INCITS 38:2009 (supersedes FIPS 5-2) USPS 2-letter codes used by the United States Postal Service USCG 2-letter codes used by the United States Coast Guard (bold red text shows differences between ANSI and USCG) Abbreviations: GPO Older variable-length official US Government Printing Office abbreviations AP Abbreviations from the AP Stylebook (bold red text shows differences between GPO and AP).4","Codes: ISO ISO 3166 codes (2-letter, 3-letter, and 3-digit codes from ISO 3166-1; 2+2-letter codes from ISO 3166-2) ANSI 2-letter and 2-digit codes from the ANSI standard INCITS 38:2009 (supersedes FIPS 5-2) USPS 2-letter codes used by the United States Postal Service USCG 2-letter codes used by the United States Coast Guard (bold red text shows differences between ANSI and USCG) Abbreviations: GPO Older variable-length official US Government Printing Office abbreviations AP Abbreviations from the AP Stylebook (bold red text shows differences between GPO and AP).5","Codes: ISO ISO 3166 codes (2-letter, 3-letter, and 3-digit codes from ISO 3166-1; 2+2-letter codes from ISO 3166-2) ANSI 2-letter and 2-digit codes from the ANSI standard INCITS 38:2009 (supersedes FIPS 5-2) USPS 2-letter codes used by the United States Postal Service USCG 2-letter codes used by the United States Coast Guard (bold red text shows differences between ANSI and USCG) Abbreviations: GPO Older variable-length official US Government Printing Office abbreviations AP Abbreviations from the AP Stylebook (bold red text shows differences between GPO a

5.3  Ilość śmiertelnych interwencji w przeliczeniu na 1000 mieszkańców każdego ze stanów

In [37]:
import pandas as pd
crim_on_1000 = pd.read_excel('crime_on_1000.xlsx')
crim_on_1000.head()

,stat,amount_crimes,population in state,crime_on_1000
0,AK,39,710231,0.054912
1,AL,104,4779736,0.021759
2,AR,83,2915918,0.028464
3,AZ,253,6392017,0.039581
4,CA,797,37254523,0.021393
